In [1]:
import os

import numpy as np
import pandas as pd
import xarray as xr
import dask.dataframe as dd

import geopandas as gpd
from shapely.geometry import Polygon

%matplotlib inline
from matplotlib import pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import mitequinox.utils as ut
import mitequinox.parcels as pa

INFO: Compiled ParcelsRandom ==> /dev/shm/pbs.4009371.datarmor0/parcels-502090/libparcels_random_bc1c98e4-8d91-4e7e-887d-98e20f35f5a6.so


In [8]:
from dask.distributed import Client, LocalCluster
False
if True:
    from dask_jobqueue import PBSCluster
    cluster = PBSCluster()
    w = cluster.scale(jobs=2)
else:    
    cluster = LocalCluster()
client = Client(cluster)
client

Client Scheduler: tcp://10.148.0.110:40008 Dashboard: http://10.148.0.110:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [2]:
root_dir = '/home1/datawork/aponte/parcels/'
run_name = 'global_extra_T365j_dt1j_dij50'

#p = pa.parcels_output(root_dir+run_name, parquets=['time'])
#df = p.df['time']
#f.head()

In [3]:
tile_dir = os.path.join(root_dir+run_name, 'tiling/')

In [4]:
import pickle
with open(os.path.join(tile_dir, "various.p"), "rb") as f:
            various = pickle.load(f)

In [5]:
D = {}
for t in ["tiles", "boundaries"]:
    D[t] = [
           (
            slice(
                various["slices"]["i_start_" + t].loc[i],
                various["slices"]["i_end_" + t].loc[i],
                ),
            slice(
                various["slices"]["j_start_" + t].loc[i],
                various["slices"]["j_end_" + t].loc[i],
                ),
            )
            for i in range(various['N_tiles'])
            ]

AttributeError: 'DataFrame' object has no attribute '_data'

In [ ]:
t_start = df.index.min().compute()
t_end = df.index.max().compute()
delta_t = (t_end - t_start)/pd.Timedelta('1D') # in days

print(t_start, t_end, delta_t)

In [ ]:
# geographical bins
dl = 2.
lon_bins = np.arange(-180.,180., dl)
lat_bins = np.arange(-90.,90., dl)

# temporal bins
#t_bins = pd.date_range(t_start, t_end, freq='1M')
t_bins = pd.date_range(t_start, t_end, freq='1MS') # starts at the beginning of each month
cut = lambda df, bins: pd.cut(df.index.to_series(), bins=bins)

In [ ]:
df['lon'] = (df['lon']+180)%360 - 180
df['lon_cut'] = df['lon'].map_partitions(pd.cut, lon_bins)
df['lat_cut'] = df['lat'].map_partitions(pd.cut, lat_bins)
df['time_cut'] = df.map_partitions(cut, t_bins)

In [ ]:
df = df.persist()

In [ ]:
def add_geodata(df):

    df['lon'] = df.apply(lambda r: r.name[0].mid, axis=1)
    df['lat'] = df.apply(lambda r: r.name[1].mid, axis=1)

    def build_polygon(r):
        lon0, lon1 = r.name[0].left, r.name[0].right
        lat0, lat1 = r.name[1].left, r.name[1].right
        return Polygon([[lon0, lat0],[lon1, lat0], [lon1, lat1], [lon0, lat1]])

    df['Coordinates'] = df.apply(build_polygon, axis=1)
    df = gpd.GeoDataFrame(df, geometry='Coordinates', crs='EPSG:4326')
    df['area'] = df.to_crs(crs = 'epsg:3857').area /1e6 / 1e4 # 100km^2 units
    
    return df

In [ ]:
means = (df
         .groupby([df.lon_cut, df.lat_cut])
         .mean()
         .compute()
         ['z']#.rename('count')
         .to_frame()
        )

# normalize by time window length
#count = count/delta_t
#
#count = add_geodata(count)
#count['density'] = count['count']/count['area']

#ds = (count
#      .drop(columns=['Coordinates'])
#      .set_index(['lon', 'lat'])
#     .to_xarray()
#    )